### Define Pattern

In [45]:
from pyparsing import *
import numpy as np

# add support for ground input '1'b0'
gnd = Literal("1'b0")
vcc = Literal("1'b1")
identifier = Word(alphas+"_", alphanums+"_"+'['+']') | gnd |vcc
input_port = Suppress("input") + delimitedList(identifier) + Suppress(";")
output_port = Suppress("output") + delimitedList(identifier) + Suppress(";")
# wire_list = Suppress("wire") + delimitedList(identifier) + Suppress(";")
# gate_type = oneOf("not and or nand nor xor")
port = Group(Suppress('.') + identifier + Suppress("(") + identifier + Suppress(")"))
gate = Group(identifier + identifier) + Suppress("(") + delimitedList(port) + Suppress(");")
# module_title = Suppress("module") + identifier + Suppress("(") + delimitedList(identifier) + Suppress(");")
# module = module_title + input_port + output_port + Optional(wire_list) + OneOrMore(gate) + Suppress("endmodule")


### Standard Cell Library Parser

In [46]:
with open('/usr/local/share/qflow/tech/osu035/osu035_stdcells.v', 'rt') as fh:
    code_std_cells = fh.read()

module_title = Suppress("module") + identifier + Suppress("(") + delimitedList(identifier) + Suppress(");")
module_list = module_title.searchString(code_std_cells)
len(module_list)

35

### Get the netlist from file

In [50]:
import os
dir = 'verilogs'
for filename in os.listdir(dir):
    
    if filename.endswith(".v"):
        with open(dir+'/'+filename, 'rt') as fh:
            code = fh.read()
        #print(filename)
        input_port_list = input_port.searchString(code)
        output_port_list = output_port.searchString(code)
        gate_list = gate.searchString(code)

        #print(len(output_port_list))
        #print(len(input_port_list))
        #print(len(gate_list))
        gate_types={}
        for i in range(len(module_list)):
        #     print(module_list[i][0])
            gate_types[module_list[i][0]] = i+2

        gate_dict = {} #gate name mapped to unique integers
        gate_type_dict = {}
        #define gate types
        #input_port = 0
        #output_port = 1
        #other gates start from 2


        #Add gnd pin at the beginning:
        gate_dict["1'b0"] = 0
        gate_type_dict[0] = 0
        gate_dict["1'b1"] = 1
        gate_type_dict[1] = 0

        #Add input ports to gate_dict and gate_type_dict:
        for idx,in_port in enumerate(input_port_list):
            gate_dict[in_port[0]] = idx+1
            gate_type_dict[idx+1] = 0

        #Add gates to gate_dict: position[2] is output
        for idx, a_gate in enumerate(gate_list):
        #     gate_dict[a_gate[0][1]] = idx + len(input_port_list)
            gate_dict[a_gate[-1][1]] = idx + len(input_port_list) +2
            gate_type_dict[idx + len(input_port_list)+1] = gate_types[a_gate[0][0]]
        #     print(idx + len(input_port_list)+1)

        #Add output ports to gate_type_dict:
        for idx, a_gate in enumerate(output_port_list):
        #     gate_dict[a_gate[0][1]] = idx + len(input_port_list)
            gate_type_dict[idx + len(input_port_list)+len(gate_list)+1] = 1
        #     print(idx + len(input_port_list)+1)
        gate_type_dict_array=[]
        for idx in range (len(gate_type_dict)):
            gate_type_dict_array.append(gate_type_dict[idx])
        #print(gate_dict)
        node_list = []
        edge_list = []
        #Add input ports and gates to node_list:
        for idx in range(len(gate_dict)):
            node_list.append(idx) 

        #Add connections to edge_list:
        for a_gate in gate_list:
            #print("GATE:",a_gate[1])
            for connection in a_gate[1:-1]:
                #print(connection)
                edge_list.append((gate_dict[connection[1]],gate_dict[a_gate[-1][1]]))

        existing_length = len(node_list)

        #Add output ports to node_list:
        for idx,out_port in enumerate(output_port_list):
            gate_type_dict[idx+existing_length] = 1 #gate type is output port
            node_list.append(idx+existing_length)
            edge_list.append((gate_dict[out_port[0]],idx+existing_length))    

        len(node_list),len(edge_list)

        filename = fh.name.split('.')
        
        filename = filename[0].split('/')
        print(filename[1])
        np.savetxt("training_data/"+filename[1]+"gate_type.csv", gate_type_dict_array, delimiter=",", fmt='%s',header='')
        np.savetxt("training_data/"+filename[1]+'node_list.csv', node_list, delimiter=",", fmt='%s',header='')
        np.savetxt("training_data/"+filename[1]+'edge_list.csv', edge_list, delimiter=",", fmt='%s',header='')

        if(filename[1].find("rca")>=0):
            graph_label = np.ones(len(node_list),np.int8)
            np.savetxt("training_data/"+filename[1]+"graph_label.csv", graph_label, delimiter=",", fmt='%s',header='')
        if(filename[1].find("cla")>=0):
            graph_label = np.zeros(len(node_list),np.int8)
            np.savetxt("training_data/"+filename[1]+"graph_label.csv", graph_label, delimiter=",", fmt='%s',header='')
        if(filename[1].find("csa")>=0):
            graph_label = np.ones(len(node_list),np.int8)
            graph_label = graph_label*2;
            np.savetxt("training_data/"+filename[1]+"graph_label.csv", graph_label, delimiter=",", fmt='%s',header='')
        if(filename[1].find("csa")>=0):
            graph_label = np.ones(len(node_list),np.int8)
            graph_label = graph_label*2;
            np.savetxt("training_data/"+filename[1]+"graph_label.csv", graph_label, delimiter=",", fmt='%s',header='')

csa_12bit
rca_14bit
csa_25bit
csa_41bit
rca_9bit
cla_52bit
rca_60bit
rca_51bit
cla_30bit
rca_46bit
csa_43bit
cla_28bit
csa_5bit
cla_27bit
cla_46bit
cla_12bit
csa_27bit
csa_10bit
csa_36bit
rca_22bit
cla_11bit
csa_16bit
rca_40bit
cla_18bit
cla_42bit
csa_45bit
csa_32bit
cla_14bit
csa_39bit
rca_52bit
cla_39bit
cla_37bit
csa_28bit
rca_49bit
rca_13bit
csa_30bit
cla_41bit
csa_47bit
csa_35bit
csa_18bit
cla_54bit
rca_43bit
cla_23bit
cla_25bit
cla_44bit
csa_52bit
rca_20bit
cla_58bit
csa_22bit
rca_29bit
csa_13bit
csa_4bit
rca_64bit
rca_44bit
cla_64bit
cla_8bit
cla_33bit
cla_57bit
csa_57bit
rca_19bit
csa_59bit
rca_62bit
csa_54bit
rca_59bit
cla_16bit
cla_26bit
cla_6bit
cla_45bit
cla_62bit
cla_38bit
cla_51bit
cla_47bit
rca_31bit
rca_58bit
rca_5bit
rca_57bit
csa_42bit
csa_20bit
cla_17bit
rca_54bit
csa_17bit
cla_4bit
cla_60bit
rca_24bit
rca_33bit
cla_40bit
cla_10bit
cla_32bit
cla_9bit
rca_21bit
cla_5bit
cla_13bit
rca_61bit
rca_47bit
cla_50bit
rca_25bit
rca_23bit
rca_15bit
csa_44bit
rca_27bit
csa_8bit


### Assign Gate Type:
reserve '0' for input and '1' for output.

### Building Graph Components

['carry_lookahead_adder_4bit', 'rtlbb', 'v']
